In [2]:
# Jupyter notebook used for the Eathical website 
# Tatum van Schooneveld, Liv Komen, Moad Matoug and Jesse Brouwer 

In [ ]:
import pandas as pd
import re

# milieu is the dutch word for environment

milieu = pd.read_excel(
    "20210209_Database milieubelasting voedingsmiddelen v1def.xlsx",
    sheet_name="tot-en-met-consumptie",
    header=2)


nevo = pd.read_excel("NEVO2023_v8.0.xlsx")


In [3]:
# Merge the two RIVM dasets 
merged = pd.merge(
    milieu,
    nevo,
    how="inner",        
    left_on="NEVO code",
    right_on="NEVO-code")

In [4]:
# Exporting
#merged.to_excel("Merged_Milieu_NEVO.xlsx", index=False)
#print("Samenvoeging voltooid — alle milieugegevens behouden.")

In [5]:
# Check the shape

print(merged.shape)

(225, 157)


In [8]:
# The Nevo dataset is per 100g of food and the environment dataset is per 1 kg of food. 
# To make the amount of food consistent and because 100 grams of food is more logical for consumption,
# the environment data is devided by 10. 

df = pd.read_excel("Merged_Milieu_NEVO.xlsx")

for col in ["kg CO2 eq", "kg SO2 eq", "kg P eq", "kg N eq", "m2a crop eq", "m3"]:
    df[col] = df[col] / 10  



In [9]:
# Again exporting 
#df.to_excel("Merged_Milieu_NEVO_per100g.xlsx", index=False)

In [9]:
# Manually removed the nutrition value columns, adjusted the names of food products, 
# and added simple English names.

In [15]:
# The cleaned and final dataset 

Cleaned_Data = pd.read_csv("DataClean.csv", sep = ";", encoding= "ISO-8859-1", engine = "python")

In [11]:
# Checking if it is the right dataset

Cleaned_Data.head()

,EnglishEasy,Global warming kg CO2,Terrestrial acidification kg SO2,Freshwater eutrophication kg P,Marine eutrophication kg N,Land use m2a crop,Water consumption m3,NEVO code,Food group,Food name,...,Vitamin D (µg),Vitamin E (mg),Vitamin K (µg),Vitamin B1 (mg),Vitamin B2 (mg),Vitamin B6 (mg),Vitamin B12 (µg),Vitamin B3 (mg),Vitamin B3 (mg).1,Vitamin C (mg)
0,Original,kg CO2 eq,kg SO2 eq,kg P eq,kg N eq,m2a crop eq,m3,NEVO code,Food group,Engelse naam/Food name,...,VITD (µg),VITE (mg),VITK (µg),THIA (mg),RIBF (mg),VITB6 (mg),VITB12 (µg),NIAEQ (mg),NIA (mg),VITC (mg)
1,NaN,"0,615258128","0,001914273","5,56E-05","0,000533716","0,264589524","0,008916192",1456,Potatoes and tubers,Chips pre-fried frozen unprepared,...,0,"0,6","0,2","0,07","0,03","0,172",0,"1,6","1,2",17
2,0,"0,108709902","0,000677535","9,44E-06","0,000141874","0,052204901","0,001018343",121,Potatoes and tubers,Potatoes mashed fresh prep with semi-skimmed m...,...,"0,1","0,4","0,4","0,06","0,06","0,145","0,12","1,3","0,8",7
3,1,"0,092314133","0,00047592","1,17E-05","0,000119007","0,035034618","0,001402564",982,Potatoes and tubers,Potatoes without skin boiled,...,0,"0,1",NaN,"0,06","0,01","0,187",0,"1,4","1,1",9
4,2,"0,070714556","0,000224979","1,13E-05","5,45E-05","0,026152547","0,001090175",390,Alcoholic beverages,Beer pilsner,...,0,0,NaN,0,"0,03","0,055","0,03",1,1,0


In [16]:
# Print the column names so they can be copied into the knn code

Cleaned_Data.columns

Index(['EnglishEasy', 'Global warming kg CO2',
       'Terrestrial acidification kg SO2', 'Freshwater eutrophication kg P ',
       'Marine eutrophication kg N ', 'Land use m2a crop',
       'Water consumption m3', 'NEVO code', 'Food group', 'Food name',
       'Energy (kJ)', 'Energy (kcal)', 'Protein (g)', 'Fat (g)',
       'Fatty acids (g)', 'Carbohydrates (g)', 'Sugars (g)', 'Fiber (g)',
       'Alcohol (g)', ' Chloride (mg)', 'Sodium (mg)', 'Potassium (mg)',
       'Calcium (mg)', 'Phosphorus (mg)', 'Magnesium (mg)', 'Iron (mg)',
       'Copper (mg)', 'Selenium (µg)', 'Zinc(mg)', 'Iodine (µg)',
       'Vitamin D (µg)', 'Vitamin E (mg)', 'Vitamin K (µg)', 'Vitamin B1 (mg)',
       'Vitamin B2 (mg)', 'Vitamin B6 (mg)', 'Vitamin B12 (µg)',
       'Vitamin B3 (mg)', 'Vitamin B3 (mg).1', 'Vitamin C (mg)'],
      dtype='object')

In [12]:
# Necessary libraries for knn algorithm

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# Nutrition columns that assess whether products are similar
nutrient_cols = ['Energy (kJ)', 'Energy (kcal)', 'Protein (g)', 'Fat (g)',
       'Fatty acids (g)', 'Carbohydrates (g)', 'Sugars (g)', 'Fiber (g)',
       'Alcohol (g)', ' Chloride (mg)', 'Sodium (mg)', 'Potassium (mg)',
       'Calcium (mg)', 'Phosphorus (mg)', 'Magnesium (mg)', 'Iron (mg)',
       'Copper (mg)', 'Selenium (µg)', 'Zinc(mg)', 'Iodine (µg)',
       'Vitamin D (µg)', 'Vitamin E (mg)', 'Vitamin K (µg)', 'Vitamin B1 (mg)',
       'Vitamin B2 (mg)', 'Vitamin B6 (mg)', 'Vitamin B12 (µg)',
       'Vitamin B3 (mg)', 'Vitamin B3 (mg).1', 'Vitamin C (mg)']

# Save results
results = []

# Iterate by food group
for group_name, group_df in Cleaned_Data.groupby('Food group'):
    if len(group_df) < 2:
        continue 

    # Prepare the Data
    X = group_df[nutrient_cols].apply(pd.to_numeric, errors="coerce").fillna(0)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    nn = NearestNeighbors(n_neighbors=min(4, len(group_df)), metric="euclidean")
    nn.fit(X_scaled)

    names = group_df['Food name'].fillna("Unknown").tolist()

    # For every product in the group
    for i, name in enumerate(names):
        distances, indices = nn.kneighbors([X_scaled[i]])
        for rank, (dist, idx) in enumerate(zip(distances[0][1:], indices[0][1:]), start=1):
            results.append({
                "Food group": group_name,
                "Product": name,
                "Vergelijkbaar product": names[idx],
                "Afstand": dist
            })

# Making a DataFrame
df_results = pd.DataFrame(results)


In [14]:
# Seeing what happened 

df_results.head(100)

,Food group,Product,Vergelijkbaar product,Afstand
0,Alcoholic beverages,Beer pilsner,Wine white dry,5.130574
1,Alcoholic beverages,Beer pilsner,Wine red,5.323618
2,Alcoholic beverages,Beer pilsner,Wine rose,5.755354
3,Alcoholic beverages,Gin young Dutch,Wine white dry,6.469972
4,Alcoholic beverages,Gin young Dutch,Wine rose,6.617582
...,...,...,...,...
95,Cold meat cuts,Beef smoke-dried,Bacon rasher streaky,6.343703
96,Cold meat cuts,Chicken (processed meat product),Ham shoulder medium fat boiled,5.001085
97,Cold meat cuts,Chicken (processed meat product),Ham lean boiled,5.111829
98,Cold meat cuts,Chicken (processed meat product),Bacon rasher streaky,5.550637


In [ ]:
# Again exporting 
#df_results.to_csv("vergelijk_goed.csv", index=False)